In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv("health_econ_raw/raw_data.csv")

print("Raw shape:", df.shape)
df.head()


Raw shape: (8984, 105)


,R0000100,R0536401,R0536402,R1200300,R1235800,R2558800,R3880300,R4893600,R4893700,R4893800,...,U4370905,U4370906,U4958600,U5862900,U5862901,U5862902,U5862903,U5862904,U5862905,U5862906
0,1,9,1981,1,1,1,1,3,3,3,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,2,7,1982,1,1,1,1,4,3,3,...,0,0,0,0,0,0,0,0,0,0
2,3,9,1983,1,1,1,1,4,2,4,...,0,0,7,1,1,1,0,3,0,1
3,4,2,1981,1,1,1,1,4,2,4,...,0,0,3,0,0,0,3,0,0,0
4,5,10,1982,1,1,1,1,4,2,2,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns = (
    df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
        .str.replace("~", "_")
)

# ================================================================
# 3. Make names readable using pattern matching
# ================================================================


In [7]:


rename_dict = {
    "id": "id",
    "r0536401": "birth_month",
    "r0536402": "birth_year",

    # Regions
    "r1200300": "region_1997",
    "r2558800": "region_1998",
    "r3880300": "region_1999",
    "r5459400": "region_2000",
    "r7222400": "region_2001",
    "s1535500": "region_2002",
    "s2005400": "region_2003",
    "s3805700": "region_2004",
    "s5405600": "region_2005",
    "s7506100": "region_2006",
    "t0009400": "region_2007",

    # Income
    "r5464100": "income_2000",
    "r7227800": "income_2001",
    "s1541700": "income_2002",
    "s2011500": "income_2003",
    "s3812400": "income_2004",
    "s5412800": "income_2005",
    "s7513700": "income_2006",
    "t0014100": "income_2007",
    "t2016200": "income_2008",
    "t3606500": "income_2009",
    "t5206900": "income_2010",

    # MSA
    "r5473500": "msa_2000",
    "r7237200": "msa_2001",
    "s1552500": "msa_2002",
    "s2022300": "msa_2003",
    "s3823200": "msa_2004",
    "s5423200": "msa_2005",
    "s7525300": "msa_2006",
    "t0025600": "msa_2007",
    "t2020500": "msa_2008",
    "t3611200": "msa_2009",
    "t5211600": "msa_2010",

    # Urban-Rural
    "r5484100": "urban_rural_2000",
    "r7248400": "urban_rural_2001",
    "s1564300": "urban_rural_2002",
    "s2034400": "urban_rural_2003",
    "s3835800": "urban_rural_2004",
    "s5436300": "urban_rural_2005",
    "s7537100": "urban_rural_2006",
    "t0033700": "urban_rural_2007",
    "t2021300": "urban_rural_2008",
    "t3612000": "urban_rural_2009",
    "t5211900": "urban_rural_2010",

    # Migration history
    "r5532200": "migrate_01_2000",
    "r5532300": "migrate_02_2000",
    "r5532400": "migrate_03_2000",
    "r5532500": "migrate_04_2000",
    "r5532600": "migrate_05_2000",
    "t5200700": "migrate_01_2010",
    "t5200800": "migrate_02_2010",
    "t5200900": "migrate_03_2010",
    "t5201000": "migrate_04_2010",
    "t5201100": "migrate_05_2010",
    "t5201200": "migrate_06_2010",

    # Mental health (YSAQ)
    "r4893600": "nervous_2000",
    "r4893700": "calm_2000",
    "r4893800": "down_2000",
    "r4893900": "happy_2000",
    "r4894000": "depressed_2000",

    "s0920800": "nervous_2002",
    "s0920900": "calm_2002",
    "s0921000": "down_2002",
    "s0921100": "happy_2002",
    "s0921200": "depressed_2002",

    "s4681900": "nervous_2004",
    "s4682000": "calm_2004",
    "s4682100": "down_2004",
    "s4682200": "happy_2004",
    "s4682300": "depressed_2004",

    "s8332300": "nervous_2006",
    "s8332400": "calm_2006",
    "s8332500": "down_2006",
    "s8332600": "happy_2006",
    "s8332700": "depressed_2006",

    "t2782600": "nervous_2008",
    "t2782700": "calm_2008",
    "t2782800": "down_2008",
    "t2782900": "happy_2008",
    "t2783000": "depressed_2008",

    "t6143700": "nervous_2010",
    "t6143800": "calm_2010",
    "t6143900": "down_2010",
    "t6144000": "happy_2010",
    "t6144100": "depressed_2010",

    # CESD scores
    "u3455400": "cesd_2019",
    "u4958600": "cesd_2021",

    # CESD items
    "u4370900": "cesd_item1_2019",
    "u4370901": "cesd_item2_2019",
    "u4370902": "cesd_item3_2019",
    "u4370903": "cesd_item4_2019",
    "u4370904": "cesd_item5_2019",
    "u4370905": "cesd_item6_2019",
    "u4370906": "cesd_item7_2019",

    "u5862900": "cesd_item1_2021",
    "u5862901": "cesd_item2_2021",
    "u5862902": "cesd_item3_2021",
    "u5862903": "cesd_item4_2021",
    "u5862904": "cesd_item5_2021",
    "u5862905": "cesd_item6_2021",
    "u5862906": "cesd_item7_2021",
}

df = df.rename(columns=rename_dict)
df.to_csv("health_econ_raw/clean_columns.csv", index=False)
print("Saved as clean_columns.csv")


Saved as clean_columns.csv


In [8]:
df = pd.read_csv("health_econ_raw/clean_columns.csv")

# ================================================================
# 2. Identify year-specific columns
# ================================================================

# Region columns look like region_1997, region_1998, ...
region_cols = [c for c in df.columns if c.startswith("region_")]

# Mental health (depressed_xxxx)
mh_cols = [c for c in df.columns if c.startswith("depressed_")]

# Controls
income_cols = [c for c in df.columns if c.startswith("income_")]
msa_cols = [c for c in df.columns if c.startswith("msa_")]
urban_cols = [c for c in df.columns if c.startswith("urban_rural_")]

# ================================================================
# 3. Reshape from wide → long
# ================================================================

# We first melt each block, then merge them all
def melt_panel(df, cols, varname):
    long = df[["id"] + cols].melt(
        id_vars="id", 
        value_vars=cols,
        var_name=f"{varname}_var",
        value_name=varname
    )
    # extract year from the column name
    long["year"] = long[f"{varname}_var"].str.extract(r"(\d{4})").astype(int)
    long = long.drop(columns=[f"{varname}_var"])
    return long

long_region = melt_panel(df, region_cols, "region")
long_mh = melt_panel(df, mh_cols, "depressed")
long_income = melt_panel(df, income_cols, "income")
long_msa = melt_panel(df, msa_cols, "msa")
long_urban = melt_panel(df, urban_cols, "urban_rural")

# Merge all long datasets on id + year
long = long_region.merge(long_mh, on=["id","year"], how="left")
long = long.merge(long_income, on=["id","year"], how="left")
long = long.merge(long_msa, on=["id","year"], how="left")
long = long.merge(long_urban, on=["id","year"], how="left")

# ================================================================
# 4. Sort and create lagged region to detect moves
# ================================================================
long = long.sort_values(["id", "year"])

# region_{t-1}
long["region_lag"] = long.groupby("id")["region"].shift(1)

# Did region change?
long["moved"] = (long["region"] != long["region_lag"]).astype(int)
long.loc[long["region_lag"].isna(), "moved"] = 0  # first year cannot be move

# Only count region moves (ignore missing)
long["move_event"] = np.where(
    (long["moved"] == 1) & (~long["region_lag"].isna()),
    1,
    0
)

# ================================================================
# 5. Count number of moves per person
# ================================================================
moves_per_id = long.groupby("id")["move_event"].sum().reset_index()
moves_per_id.columns = ["id", "num_moves"]

# Keep only one-time movers
one_move_ids = moves_per_id[moves_per_id["num_moves"] == 1]["id"]

clean_long = long[long["id"].isin(one_move_ids)]

# ================================================================
# 6. Get the move year for each ID
# ================================================================
move_years = (
    clean_long[clean_long["move_event"] == 1]
    .groupby("id")["year"]
    .first()
    .reset_index()
)
move_years.columns = ["id", "move_year"]

clean_long = clean_long.merge(move_years, on="id", how="left")

# ================================================================
# 7. Create event-time variable (year - move_year)
# ================================================================
clean_long["event_time"] = clean_long["year"] - clean_long["move_year"]

# Optional: Keep only event_time in [-3, +3]
clean_long = clean_long[(clean_long["event_time"] >= -3) & 
                        (clean_long["event_time"] <= 3)]

# ================================================================
# 8. Save final event-study dataset
# ================================================================
clean_long.to_csv("health_econ_raw/event_study_dataset.csv", index=False)
print("Saved event-study dataset!")

Saved event-study dataset!
